In [17]:
import sys
import numpy as np
if '../' not in sys.path:
    sys.path.append('../')
import gym

In [8]:
from lib.envs.gridworld import GridworldEnv

In [145]:
env = GridworldEnv()
print env.nS
v = np.zeros(env.nS)
print v
env.shape
for s in env.P:
    print s
print env.P[1]
print env.P[0][1]
print env.P[0][0]
print env.P[1][1]
print env.action_space
#print np.ones([env.nS, env.nA])/env.nA

16
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
{0: [(1.0, 1, -1.0, False)], 1: [(1.0, 2, -1.0, False)], 2: [(1.0, 5, -1.0, False)], 3: [(1.0, 0, -1.0, True)]}
[(1.0, 0, 0.0, True)]
[(1.0, 0, 0.0, True)]
[(1.0, 2, -1.0, False)]
Discrete(4)


In [ ]:
print env.action_space.sample

In [149]:
def policy_eval(policy, env, discount_factor=1.0, theta=0.00001):
    """
    Evaluate a policy given an environment and a full description of the environment's dynamics.
    
    Args:
        policy: [S, A] shaped matrix representing the policy.
        env: OpenAI env. env.P represents the transition probabilities of the environment.
            env.P[s][a] is a (prob, next_state, reward, done) tuple.
        theta: We stop evaluation one our value function change is less than theta for all states.
        discount_factor: lambda discount factor.
    
    Returns:
        Vector of length env.nS representing the value function.
                
    """
    
    V = np.zeros(env.nS)
    while True:
        V_next = np.zeros(env.nS)
        # check each state
        for s in env.P:
            v = 0
            # check each action
            for a, action_prob in enumerate(policy[s]):
                # get values for each possible next state given action a
                for next_state_prob, next_state, reward, done in env.P[s][a]:
                    V_next[s] += action_prob * next_state_prob * (reward + discount_factor * V[next_state])
        if np.all((V - V_next) < theta):
            break
        V = V_next
    return V

    """
    my initial attempt, works but maybe not as clean
    
    
    V = np.zeros(env.nS)
    i = 0
    while True:
        V2 = np.zeros(env.nS)
        for s in env.P.keys():
            new_value = 0
            for a in env.P[s]:
                next_state = env.P[s][a][0][1]
                next_state_value = V[next_state]
                r = env.P[s][a][0][2]
                p = policy[s][a]
                new_value += p*(r+next_state_value)
            V2[s] = new_value
        if np.all((V-V2) < theta):
            break
        V = V2
    return V
    """
        
        

In [150]:
random_policy = np.ones([env.nS, env.nA])/env.nA
v = policy_eval(random_policy, env)
print v

[  0.         -13.99988715 -19.99983277 -21.99981286 -13.99988715
 -17.99985268 -19.99983389 -19.99983277 -19.99983277 -19.99983389
 -17.99985268 -13.99988715 -21.99981286 -19.99983277 -13.99988715   0.        ]


In [152]:
expected_v = np.array([0, -14, -20, -22, -14, -18, -20, -20, -20, -20, -18, -14, -22, -20, -14, 0])
np.testing.assert_array_almost_equal(v, expected_v, decimal=2)